## SISTEMA DE RECOMENDACION:

## EDA

In [43]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from surprise.model_selection import train_test_split, GridSearchCV,train_test_split
from surprise import Dataset, Reader, SVD,KNNWithMeans,NMF
from surprise.prediction_algorithms.matrix_factorization import SVD
from surprise import accuracy

In [44]:
df_usuarios = pd.read_csv("df_ML.csv")
df_usuarios.head()

,userid,rating,movieid,title
0,1,1.0,as680,The English Civil War
1,583,4.5,as680,The English Civil War
2,765,5.0,as680,The English Civil War
3,2116,3.0,as680,The English Civil War
4,2143,3.0,as680,The English Civil War


In [45]:
df_usuarios.shape 

(10995634, 4)

In [46]:
#Paso la columna movieid a un valor categórico numérico

le = LabelEncoder()
df_usuarios['movieid'] = le.fit_transform(df_usuarios['movieid'])
df_usuarios.head()

,userid,rating,movieid,title
0,1,1.0,6445,The English Civil War
1,583,4.5,6445,The English Civil War
2,765,5.0,6445,The English Civil War
3,2116,3.0,6445,The English Civil War
4,2143,3.0,6445,The English Civil War


In [47]:
df_usuarios.duplicated().value_counts()

False    10995634
dtype: int64

In [48]:
#Cantidad de películas totales en el dataframe
df_usuarios["movieid"].nunique()

22998

In [49]:
df_usuarios_eda = df_usuarios.drop(columns=["title"])

In [50]:
df_usuarios_eda.to_pickle("https://www.dropbox.com/s/up7v9kce3son55b/dataframe_EDA.pkl?dl=1")

## ML PROCESS

#### `MODELO CON ALGORITMO SVD (Singular Value Decomposition):`


 <div style="width: 120px; height: 35px; background-color: #E2E2E2; border-radius: 10px; display: flex; justify-content: center; align-items: center;">
    <span style="font-size: 18px; font-weight: bold; color: #000000;">Información</span>
</div>El algoritmo SVD de la librería surprise utiliza el enfoque de filtrado colaborativo (user-user) para hacer recomendaciones personalizadas para un usuario dado. En particular, utiliza el algoritmo SVD para factorizar la matriz de calificaciones de los usuarios (que tiene filas correspondientes a los usuarios, columnas correspondientes a las películas y elementos correspondientes a las calificaciones de los usuarios para esas películas) en dos matrices de factores latentes: una matriz de usuarios y una matriz de películas. Luego, utiliza estas matrices para hacer predicciones de calificación para películas que el usuario aún no ha visto.

Para encontrar a los usuarios más similares al usuario dado, el algoritmo calcula las similitudes de coseno entre el vector de preferencias del usuario dado (es decir, sus calificaciones para todas las películas) y los vectores de preferencias de todos los demás usuarios en el conjunto de datos. Luego, se selecciona un subconjunto de usuarios con las similitudes más altas y se utiliza la información de calificación de estas películas para hacer predicciones para el usuario dado.

En resumen, el algoritmo utiliza información sobre las películas vistas por el usuario dado (a través de la matriz de calificaciones) para encontrar a los usuarios más similares a él y hacer predicciones de calificación personalizadas para las películas no vistas por ese usuario.

In [51]:

reader = Reader(rating_scale=(1, 5))  # Fuerzo que la escala de cálculo de la estimación de salida sea de 1 a 5.Si no, el software podría adoptar otro criterio

data = Dataset.load_from_df(df_usuarios[['userid', 'movieid', 'rating']], reader)

# Separamos nuestros datos
trainset, testset = train_test_split(data, test_size=.25)

model = SVD()

#Entrenamos
model.fit(trainset)

# Predecimos
predictions = model.test(testset)


In [52]:
# Elegimos un usuario y una película para hacer una recomendación
model.predict(100,123)


# Tomaremos un usuario para hacerle una recomendación
print(model.predict(100,123))

user: 100        item: 123        r_ui = None   est = 3.39   {'was_impossible': False}


Interpretación de resultados:  
- uid : es el número de usuario para el cual se realizó la predicción.
- iid : este es id de la película seleccionada para realizar la predicción.
- est : este es el valor promedio estimado que un grupo de usuarios con características similares al usuario seleccionadoa califica la película.
- {'was_impossible': False}: el parámetro False nos indica que la predicción se pudo hacer correctamente.

METRICAS:

In [53]:
"""

Es importante tener una métrica de bondad de ajuste (coef. determinación r2) y una métrica de error.La librería surprise no trae la métrica r2 por lo tanto hay que calcularla.

"""

rmse = accuracy.rmse(predictions)
mse = accuracy.mse(predictions)

print("RMSE:", rmse)
print("MSE:", mse)


RMSE: 1.0022
MSE: 1.0044
RMSE: 1.002206964181792
MSE: 1.0044187990544833


#### Aplicando Optimización de parámetros con GridSearch

In [54]:
"""

# Definir los parámetros para el modelo SVD y crear un objeto GridSearchCV para encontrar los mejores parámetros a través de la validación cruzada:
param_grid = {'n_factors': [50, 100, 150],'n_epochs': [20, 30, 40]}   

# crear un objeto GridSearchCV           
gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=3, n_jobs=-1)

gs.fit(data)

"""


"\n\n# Definir los parámetros para el modelo SVD y crear un objeto GridSearchCV para encontrar los mejores parámetros a través de la validación cruzada:\nparam_grid = {'n_factors': [50, 100, 150],'n_epochs': [20, 30, 40]}   \n\n# crear un objeto GridSearchCV           \ngs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=3, n_jobs=-1)\n\ngs.fit(data)\n\n"

<div style="width: 120px; height: 35px; background-color: #E2E2E2; border-radius: 10px; display: flex; justify-content: center; align-items: center;">
    <span style="font-size: 18px; font-weight: bold; color: #000000;">Información</span>
</div>Parámetros de ajuste seleccionados en la grilla de Gridsearch para optimizar el modelo SVD:

- n_factors: Número de factores latentes que se utilizan para representar a los usuarios y elementos en la matriz de factores latentes. El valor predeterminado es 100. Si el valor es demasiado bajo, el modelo puede no capturar todas las relaciones importantes entre los usuarios y los elementos en los datos. Si el valor es demasiado alto, el modelo puede sobreajustarse a los datos y no generalizar bien a nuevos datos.

- n_epochs: Número de épocas (iteraciones) para entrenar el modelo. El valor predeterminado es 20. Si el valor es demasiado bajo, el modelo puede no converger a una solución óptima. Si el valor es demasiado alto, el modelo puede sobreajustarse a los datos y no generalizar bien a nuevos datos.

- lr_all: Tasa de aprendizaje para todos los parámetros del modelo. El valor predeterminado es 0.005. Controla la magnitud de las actualizaciones de parámetros durante el entrenamiento del modelo. Si el valor es demasiado alto, el modelo puede diverger y no converger a una solución óptima. Si el valor es demasiado bajo, el modelo puede tardar más en converger a una solución óptima.

- reg_all: Tasa de regularización para todos los parámetros del modelo. El valor predeterminado es 0.02. Controla la magnitud de los pesos de regularización aplicados a los parámetros del modelo durante el entrenamiento. La regularización ayuda a prevenir el sobreajuste del modelo. Si el valor es demasiado alto, el modelo puede ser demasiado restrictivo y no capturar todas las relaciones importantes en los datos. Si el valor es demasiado bajo, el modelo puede sobreajustarse a los datos y no generalizar bien a nuevos datos.


In [55]:
"""

# Métricas con GridSearch:

best_rmse = gs.best_score['rmse']
print("best_rmse: ",best_rmse)

"""

'\n\n# Métricas con GridSearch:\n\nbest_rmse = gs.best_score[\'rmse\']\nprint("best_rmse: ",best_rmse)\n\n'


#### `MODELO CON ALGORITMO KNNWithMEANS :`

In [56]:
"""

reader = Reader(rating_scale=(1, 5))  # Fuerzo que la escala de cálculo de la estimación de salida sea de 1 a 5.Si no, el software podría adoptar otro criterio

data = Dataset.load_from_df(df_usuarios[['userid', 'movieid', 'rating']], reader)

# Separamos nuestros datos
trainset, testset = train_test_split(data, test_size=.25)

model_1 = KNNWithMeans()

#Entrenamos
model_1.fit(trainset)

# Predecimos
predictions_1 = model_1.test(testset)
# Elegimos un usuario y una película para hacer una recomendación
model_1.predict(146200,2255)
# Tomaremos un usuario para hacerle una recomendación
print(model_1.predict(146200,2255))

"""

"\n\nreader = Reader(rating_scale=(1, 5))  # Fuerzo que la escala de cálculo de la estimación de salida sea de 1 a 5.Si no, el software podría adoptar otro criterio\n\ndata = Dataset.load_from_df(df_usuarios[['userid', 'movieid', 'rating']], reader)\n\n# Separamos nuestros datos\ntrainset, testset = train_test_split(data, test_size=.25)\n\nmodel_1 = KNNWithMeans()\n\n#Entrenamos\nmodel_1.fit(trainset)\n\n# Predecimos\npredictions_1 = model_1.test(testset)\n# Elegimos un usuario y una película para hacer una recomendación\nmodel_1.predict(146200,2255)\n# Tomaremos un usuario para hacerle una recomendación\nprint(model_1.predict(146200,2255))\n\n"

<div style="width: 120px; height: 35px; background-color: #E2E2E2; border-radius: 10px; display: flex; justify-content: center; align-items: center;">
    <span style="font-size: 18px; font-weight: bold; color: #000000;">Información</span>
</div> Tira error pof falta de capacidad de procesamiento y memoria para procesar una matriz (1000000 x 1000000)

En general, el algoritmo kNNwithMeans y el algoritmo SVD tienen requerimientos de recursos similares, ya que ambos deben procesar todo el conjunto de datos y realizar cálculos intensivos.

Sin embargo, hay algunos factores que pueden influir en los requerimientos de recursos de cada algoritmo en diferentes situaciones. Por ejemplo:

Tamaño del conjunto de datos: Si el conjunto de datos es muy grande, kNNwithMeans podría requerir más recursos ya que necesita calcular las similitudes entre todas las parejas de usuarios/items, lo que implica comparar cada usuario/item con todos los demás en el conjunto de datos. En cambio, SVD utiliza descomposición de matriz para reducir la complejidad del cálculo de las recomendaciones, lo que lo hace más escalable.

Densidad del conjunto de datos: Si el conjunto de datos es muy denso (es decir, la mayoría de las entradas están completas), SVD podría requerir más recursos ya que necesita realizar cálculos en todas las entradas, mientras que kNNwithMeans solo necesita calcular las similitudes entre los elementos que tienen entradas en común.

Parámetros del modelo: Los parámetros de cada modelo también pueden influir en los requerimientos de recursos. Por ejemplo, el número de factores latentes en SVD puede afectar la complejidad de los cálculos, mientras que el número de vecinos k en kNNwithMeans puede influir en la cantidad de cálculos necesarios para calcular las similitudes.

En resumen, kNNwithMeans y SVD tienen requerimientos de recursos similares en general, pero el algoritmo más adecuado dependerá del tamaño y la densidad del conjunto de datos, así como de los parámetros específicos del modelo. Es importante tener en cuenta estos factores al elegir el algoritmo adecuado para un problema de recomendación específico.

#### `MODELO CON ALGORITMO NMF:`  

<div style="width: 120px; height: 35px; background-color: #E2E2E2; border-radius: 10px; display: flex; justify-content: center; align-items: center;">
    <span style="font-size: 18px; font-weight: bold; color: #000000;">Información</span>
</div> En Surprise, NMF significa "Non-negative Matrix Factorization" (Factorización de matriz no negativa). Es un algoritmo de factorización de matrices que se utiliza para reducir la dimensionalidad de un conjunto de datos y para encontrar patrones latentes en los datos. En el contexto de la recomendación de películas, se utiliza para descomponer la matriz de valoraciones de los usuarios y las películas en dos matrices más pequeñas que representan características latentes de las películas y preferencias latentes de los usuarios. Luego, estas dos matrices se multiplican para generar una aproximación de la matriz original de valoraciones, que se utiliza para hacer recomendaciones a los usuarios.



In [57]:
# El modelo SVD optimizado es más eficiente.Se anula la ejecución de NMF porque esmuy lenta la ejecucuión de todo el código del archivo"

"""

reader = Reader(rating_scale=(1, 5))  # Fuerzo que la escala de cálculo de la estimación de salida sea de 1 a 5.Si no, el software podría adoptar otro criterio

data = Dataset.load_from_df(df_usuarios[['userid', 'movieid', 'rating']], reader)

# Separamos nuestros datos
trainset, testset = train_test_split(data, test_size=.25)

# Crear una instancia del modelo NMF
model_2 = NMF()

# Entrenar el modelo con el conjunto de entrenamiento
model_2.fit(trainset)

# Hacer predicciones para el conjunto de prueba
predictions_2 = model_2.test(testset)

# Calcular el error RMSE y MAE
rmse = accuracy.rmse(predictions_2)
mse = accuracy.mse(predictions_2)

r2 = 1 - mse / ((df_usuarios['rating'] - df_usuarios['rating'].mean())**2).mean()  

# Hacer una predicción para un usuario y una película específicos
uid = 100  # ID del usuario
iid = 123  # ID de la película
model_2.predict(uid, iid)
print(model_2.predict(uid, iid))

"""


"\n\nreader = Reader(rating_scale=(1, 5))  # Fuerzo que la escala de cálculo de la estimación de salida sea de 1 a 5.Si no, el software podría adoptar otro criterio\n\ndata = Dataset.load_from_df(df_usuarios[['userid', 'movieid', 'rating']], reader)\n\n# Separamos nuestros datos\ntrainset, testset = train_test_split(data, test_size=.25)\n\n# Crear una instancia del modelo NMF\nmodel_2 = NMF()\n\n# Entrenar el modelo con el conjunto de entrenamiento\nmodel_2.fit(trainset)\n\n# Hacer predicciones para el conjunto de prueba\npredictions_2 = model_2.test(testset)\n\n# Calcular el error RMSE y MAE\nrmse = accuracy.rmse(predictions_2)\nmse = accuracy.mse(predictions_2)\n\nr2 = 1 - mse / ((df_usuarios['rating'] - df_usuarios['rating'].mean())**2).mean()  \n\n# Hacer una predicción para un usuario y una película específicos\nuid = 100  # ID del usuario\niid = 123  # ID de la película\nmodel_2.predict(uid, iid)\nprint(model_2.predict(uid, iid))\n\n"

In [58]:
"""

print("RMSE:", rmse)
print("MSE:", mse)
print("R2:", r2)

"""

'\n\nprint("RMSE:", rmse)\nprint("MSE:", mse)\nprint("R2:", r2)\n\n'

#### Aplicando Optimización de parámetros con GridSearch

In [59]:
"""
# Definir los parámetros para el modelo NMF y crear un objeto GridSearchCV para encontrar los mejores parámetros a través de la validación cruzada:
param_grid = {'n_factors': [50, 100, 150], 'n_epochs': [20, 30, 40]}

# crear un objeto GridSearchCV           
gs1 = GridSearchCV(NMF, param_grid, measures=['rmse'], cv=3, n_jobs=-1)

# Ajustar el modelo
gs1.fit(data)

"""

"\n# Definir los parámetros para el modelo NMF y crear un objeto GridSearchCV para encontrar los mejores parámetros a través de la validación cruzada:\nparam_grid = {'n_factors': [50, 100, 150], 'n_epochs': [20, 30, 40]}\n\n# crear un objeto GridSearchCV           \ngs1 = GridSearchCV(NMF, param_grid, measures=['rmse'], cv=3, n_jobs=-1)\n\n# Ajustar el modelo\ngs1.fit(data)\n\n"

In [60]:
"""

#Métricas con Gridsearch
best_rmse_1 = gs1.best_score['rmse']
print("best_rmse: ",best_rmse_1)

"""

'\n\n#Métricas con Gridsearch\nbest_rmse_1 = gs1.best_score[\'rmse\']\nprint("best_rmse: ",best_rmse_1)\n\n'

- Se selecciona por perfomance el modelo SVD optimizado por GridSearch

#### Se guarda el modelo en una librería especializda

<div style="width: 120px; height: 35px; background-color: #E2E2E2; border-radius: 10px; display: flex; justify-content: center; align-items: center;">
    <span style="font-size: 18px; font-weight: bold; color: #000000;">Información</span>
</div> Tanto Joblib como Pickle son librerías de Python que se utilizan para guardar modelos de machine learning entrenados y otros objetos de Python. Ambas librerías son útiles para serializar y deserializar objetos de Python, lo que permite guardar y cargar modelos de forma rápida y sencilla.

Sin embargo, hay algunas diferencias entre Joblib y Pickle que pueden afectar la elección de una u otra para guardar modelos de machine learning entrenados:

Pickle es una librería estándar de Python, lo que significa que ya está instalada en la mayoría de las distribuciones de Python. Por otro lado, Joblib es una librería externa que debe ser instalada por separado.

Pickle es una librería muy versátil que puede manejar la mayoría de los tipos de objetos de Python, mientras que Joblib está especialmente diseñada para serializar y deserializar objetos grandes y complejos, como los modelos de machine learning.

Pickle puede ser más lento que Joblib al serializar objetos grandes, ya que utiliza un protocolo de serialización basado en texto. Joblib, por otro lado, utiliza un protocolo de serialización binario que puede ser más rápido.

En general, ambos Joblib y Pickle son buenas opciones para guardar modelos de machine learning entrenados. Pickle es más versátil y ya está instalado en Python, mientras que Joblib está optimizado para manejar objetos grandes y complejos, como los modelos de machine learning. La elección entre las dos librerías dependerá del tamaño y complejidad del modelo, así como de las preferencias personales.

In [62]:
from joblib import dump

# Guardar(exporta) el modelo  SVD en un archivo llamado "modelo_svd.joblib"
dump(model, 'modelo_svd.joblib')

#El archivo modelo_svd.joblib que alimenta a la consola de Gradio está en Dropbox


['modelo_svd.joblib']